## [Interactive Analysis with the Spark Shell](https://spark.apache.org/docs/latest/quick-start.html#interactive-analysis-with-the-spark-shell)

### Basic

ENG: Spark’s primary abstraction is a distributed collection of items called a Dataset. Datasets can be created from Hadoop InputFormats (such as HDFS files) or by transforming other Datasets.

TR: Spark' ın `birincil soyutlaması(primary abstraction)` `item'ların dağıtık bir koleksiyonu(distributed collection of items)` olan Datasetlerdir. Datasetler Hadoop InputFormats (HDFS dosyalarıyla) ya da diğer datasetlerden dönüştürülerek oluşturulabilir.

In [8]:
val textFile = spark.read.textFile("../README.md") //Projede bulunan README.md 
//text dosyasından yeni bir Dataset oluşturuyoruz.

// Sanırım:
// Bir Datasetten veri alma işlemine action deniyor.
// Bir Datasetin itemlarını kullanarak yeni bir Dataset oluşturmaya transform deniyor.

textFile: org.apache.spark.sql.Dataset[String] = [value: string]


In [7]:
// Bir Dataset üzerinde yapılabilecek bazı action' lar
textFile.count() // Number of items in this Dataset. TR: README.md dosyasında bulunan satır sayısı.
//Demekki item diye bahsettiği şey aslında satır sayısıymış.

res3: Long = 8


In [9]:
// Bir Dataset üzerinde yapılabilecek bazı action' lar
textFile.first() // First item in this Dataset

res5: String = ## Giriş


In [3]:
// Transform işlemi (bir datasetten yeni bir dataset oluşturma) için filter adlı bir fonksiyon kullandık. 
// İçinde Spark geçen item' lardan yeni bir tane Dataset oluşturduk.
val lineWithSpark = textFile.filter(line => line.contains("Spark"))

lineWithSpark: org.apache.spark.sql.Dataset[String] = [value: string]


In [13]:
// Transform ve action işlemlerini bu şekilde birleştirebiliyoruz.
textFile.filter(line => line.contains("Spark")).count()

res8: Long = 2


### More on Dataset Operations

In [16]:
// Daha karmaşık hesaplamalar için transform ve action işlemlerini kullanabiliriz.
// Mesela en çok kelime bulunan satırı bulmak isteyelim.
textFile.map(line => line.split(" ").size).reduce((a, b) => if (a > b) a else b)
// İlk olarak satırları integer sayılara map edip yeni bir Dataset oluşturuyoruz. reduce fonksiyonu yeni oluşan
// Dataset tarafından çağrılır ve en yüksek kelime sayısını bulur.

res11: Int = 22


In [18]:
// map ve reduce fonksiyonlarının argümanları Scala funtion literals (closures) özelliğindedir. [Python'daki iterator]
// Bu yüzden bu fonksiyonlar herhangi bir language feature ya da Scala/ Java kütüphanesinden 
// belirli feature' i alabilir. Mesela yukarıdaki işlemi daha okunabilir yapalım.
import java.lang.Math

textFile.map(line => line.split(" ").size).reduce((a,b) => Math.max(a, b))

import java.lang.Math
res13: Int = 22


In [33]:
// Tam olarak ne yaptığını  görmek istiyorum.
textFile.map(line => line.split(" ")).show() //map ile transform show, ile action

+--------------------+
|               value|
+--------------------+
|         [##, Giriş]|
|                  []|
|[Apache, Spark, i...|
|                  []|
|[~~Apache, Spark'...|
|                  []|
|                  []|
|[Scala, kullanmay...|
+--------------------+



newTextFileDataset: org.apache.spark.sql.Dataset[Array[String]] = [value: array<string>]


In [34]:
val newTextFileDataset = textFile.map(line => line.split(" ").size) // map ile transform, show ile action
newTextFileDataset.show()

+-----+
|value|
+-----+
|    2|
|    1|
|    6|
|    1|
|   22|
|    1|
|    1|
|   10|
+-----+



newTextFileDataset: org.apache.spark.sql.Dataset[Int] = [value: int]


In [37]:
newTextFileDataset.reduce((a, b) => if (a > b) a else b)

res27: Int = 22


In [3]:
// One common data flow pattern is MapReduce, as popularized by Hadoop. Spark can implement MapReduce flows easily
val wordCounts = textFile.flatMap(line => line.split(" ")).groupByKey(identity).count()

// flat map ile satır Dataset' i olan textFile' ı kelime datasetine çeviriyor. 
// groupByKey ve count ile kelimelerin sayısını elde ediliyor.

wordCounts: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


In [5]:
wordCounts.collect()

res2: Array[(String, Long)] = Array((şekilde,1), (ı,1), (notlarımı,1), (Giriş,1), (Sistemi,1), (kullanmaya,1), (olarakta,1), (~~Apache,1), (yapabiliyormuşuz.,1), (verdim.,2), (Linux,1), (kullanmaktayım.,1), (dağıtım,1), (Apache,1), (olarak,1), (bu,1), (işlemleri,1), (Ubuntu,1), (Scala,1), (yazmaya,1), (İşletim,1), (yaptım.~~,1), (PySpark,1), (ile,3), (Python,1), (Spark',1), (##,1), (Spark,1), (ilgili,1), (Jupyter,1), ("",4), (tutacağım.,1), (karar,2), (kullandım.,1), ([şu](./kurulum.md),1), (notebook,1), (Kurulumu,1))


In [10]:
// Şimdi yukarıdaki işlemleri teker teker yapalım
val wordDataset = textFile.flatMap(line => line.split(" "))
wordDataset.show()

+----------+
|     value|
+----------+
|        ##|
|     Giriş|
|          |
|    Apache|
|     Spark|
|       ile|
|    ilgili|
| notlarımı|
|tutacağım.|
|          |
|  ~~Apache|
|    Spark'|
|         ı|
|    Python|
|       ile|
|   yazmaya|
|     karar|
|   verdim.|
|   PySpark|
|kullandım.|
+----------+
only showing top 20 rows



wordDataset: org.apache.spark.sql.Dataset[String] = [value: string]


In [17]:
val workDatasetKeyCountPair = wordDataset.groupByKey(identity).count()
workDatasetKeyCountPair.show()

+-----------------+--------+
|              key|count(1)|
+-----------------+--------+
|          şekilde|       1|
|                ı|       1|
|        notlarımı|       1|
|            Giriş|       1|
|          Sistemi|       1|
|       kullanmaya|       1|
|         olarakta|       1|
|         ~~Apache|       1|
|yapabiliyormuşuz.|       1|
|          verdim.|       2|
|            Linux|       1|
|  kullanmaktayım.|       1|
|          dağıtım|       1|
|           Apache|       1|
|           olarak|       1|
|               bu|       1|
|        işlemleri|       1|
|           Ubuntu|       1|
|            Scala|       1|
|          yazmaya|       1|
+-----------------+--------+
only showing top 20 rows



workDatasetKeyCountPair: org.apache.spark.sql.Dataset[(String, Long)] = [key: string, count(1): bigint]


In [23]:
workDatasetKeyCountPair.collect()

res17: Array[(String, Long)] = Array((şekilde,1), (ı,1), (notlarımı,1), (Giriş,1), (Sistemi,1), (kullanmaya,1), (olarakta,1), (~~Apache,1), (yapabiliyormuşuz.,1), (verdim.,2), (Linux,1), (kullanmaktayım.,1), (dağıtım,1), (Apache,1), (olarak,1), (bu,1), (işlemleri,1), (Ubuntu,1), (Scala,1), (yazmaya,1), (İşletim,1), (yaptım.~~,1), (PySpark,1), (ile,3), (Python,1), (Spark',1), (##,1), (Spark,1), (ilgili,1), (Jupyter,1), ("",4), (tutacağım.,1), (karar,2), (kullandım.,1), ([şu](./kurulum.md),1), (notebook,1), (Kurulumu,1))


### Caching

Bu kısmı tam olarak anlamadım.

Spark bildiğim kadarıyla verileri RAM' a alarak işliyor. Hadoop' tan farkı buradan ortaya çıkıyor. Hadoop verileri Disk üzerinden işler Spark ise RAM üzerinden veriler üzerinden işlem yapar.

In [6]:
lineWithSpark.cache() // Şuan veriler cache' e alındı

res3: lineWithSpark.type = [value: string]


In [7]:
lineWithSpark.count()

res4: Long = 2


In [ ]:
// Self-Contained Applications diye bir kısım vardı. Ona bakmadım. Scala projesinde Spark kullanmayı gösteriyor sanırım